In [18]:
# 🔹 CELL 1: Import Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [19]:
# 🔹 CELL 2: Load Dataset
df = pd.read_csv("sonardataset.csv", header=None)

df.columns = [f"V{i}" for i in range(1, 61)] + ["Target"]

df.head()


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Target
0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_52,x_53,x_54,x_55,x_56,x_57,x_58,x_59,x_60,Y
1,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,R
2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.014,0.0049,0.0052,0.0044,R
3,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.018,0.0244,0.0316,0.0164,0.0095,0.0078,R
4,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.015,0.0085,0.0073,0.005,0.0044,0.004,0.0117,R


In [22]:
# 🔹 CELL 3: CLEAN THE DATASET (MOST IMPORTANT FIX)
# Keep only valid classes (M and R)
df = df[df["Target"].isin(["M", "R"])]

# Convert all feature columns to numeric
for col in df.columns[:-1]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with any non-numeric values
df.dropna(inplace=True)

print("Clean Dataset Shape:", df.shape)
print("\nClass Distribution:\n", df["Target"].value_counts())


Clean Dataset Shape: (208, 61)

Class Distribution:
 Target
M    111
R     97
Name: count, dtype: int64


In [23]:
# 🔹 CELL 4: Preprocessing
X_data = df.iloc[:, :-1].values
y_data = df.iloc[:, -1].values

# Encode target (M=1, R=0)
le = LabelEncoder()
y_data = le.fit_transform(y_data)

# Feature Scaling
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)

print("Training shape:", X_train.shape)
print("Testing shape :", X_test.shape)


Training shape: (166, 60)
Testing shape : (42, 60)


In [24]:
# 🔹 CELL 5: ANN MODEL – DEFAULT
ann_default = MLPClassifier(
    hidden_layer_sizes=(32,),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

ann_default.fit(X_train, y_train)


MLPClassifier(hidden_layer_sizes=(32,), max_iter=500, random_state=42)

In [25]:
#🔹 CELL 6: Evaluation – Default Model 
y_pred_default = ann_default.predict(X_test)

print("DEFAULT MODEL ACCURACY:", accuracy_score(y_test, y_pred_default))
print("\nClassification Report:\n", classification_report(y_test, y_pred_default))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_default))


DEFAULT MODEL ACCURACY: 0.9047619047619048

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.85      0.92        26
           1       0.80      1.00      0.89        16

    accuracy                           0.90        42
   macro avg       0.90      0.92      0.90        42
weighted avg       0.92      0.90      0.91        42


Confusion Matrix:
 [[22  4]
 [ 0 16]]


In [26]:
# 🔹 CELL 7: ANN MODEL – TUNED
ann_tuned = MLPClassifier(
    hidden_layer_sizes=(64, 32, 16),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=800,
    random_state=42
)

ann_tuned.fit(X_train, y_train)


MLPClassifier(hidden_layer_sizes=(64, 32, 16), max_iter=800, random_state=42)

In [27]:
# 🔹 CELL 8: Evaluation – Tuned Model
y_pred_tuned = ann_tuned.predict(X_test)

print("TUNED MODEL ACCURACY:", accuracy_score(y_test, y_pred_tuned))
print("\nClassification Report:\n", classification_report(y_test, y_pred_tuned))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_tuned))


TUNED MODEL ACCURACY: 0.9047619047619048

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.88      0.92        26
           1       0.83      0.94      0.88        16

    accuracy                           0.90        42
   macro avg       0.90      0.91      0.90        42
weighted avg       0.91      0.90      0.91        42


Confusion Matrix:
 [[23  3]
 [ 1 15]]


In [28]:
# 🔹 CELL 9: Final Comparison
print("Default ANN Accuracy :", accuracy_score(y_test, y_pred_default))
print("Tuned ANN Accuracy   :", accuracy_score(y_test, y_pred_tuned))


Default ANN Accuracy : 0.9047619047619048
Tuned ANN Accuracy   : 0.9047619047619048
